In [1]:
import pandas as pd
import numpy as np
import re

# Load data processing

In [8]:
# Read original hourly load
region = pd.read_csv('Processed Data/Reduced_Area_Region.csv')
load = pd.read_csv('Public Data/Hourly Profiles in CSV Format/Hourly Profile_Load.csv')

In [9]:
# reduce column names to Area Names
def extract_name(name):
    match = re.search(r'_(\w+)_', name)
    return match.group(1) if match else name

load.columns = [extract_name(name) for name in load.columns]

print(load.columns)

Index(['Index', 'AESO', 'AVA', 'AZPS', 'BANC', 'BCHA', 'BPAT', 'CFE', 'CHPD',
       'CIPB', 'CIPV', 'CISC', 'CISD', 'DOPD', 'EPE', 'GCPD', 'IID', 'IPFE',
       'IPMV', 'IPTV', 'LDWP', 'NEVP', 'NWMT', 'PACW', 'PAID', 'PAUT', 'PAWY',
       'PGE', 'PNM', 'PSCO', 'PSEI', 'SCL', 'SPPC', 'SRP', 'TEPC', 'TH_Malin',
       'TH_Mead', 'TH_PV', 'TIDC', 'TPWR', 'VEA', 'WACM', 'WALC', 'WAUW',
       'Unnamed: 44'],
      dtype='object')


In [10]:
# filter load data and keep 8784 hours (rows) and drop last empty column
load = load.loc[1:8784]
load = load.drop(load.columns[-1], axis=1)

# set column index as index
load = load.set_index(load.columns[0])

# load data to numeric, originally string
load = load.apply(pd.to_numeric)

In [11]:
# Create an empty DataFrame for the results
load_reduced = pd.DataFrame()

# Loop over the unique values in df_map
for value in region['Zone'].unique():
    # Get the columns that are mapped to this value
    cols = region.loc[region['Zone'] == value, 'Area Name']
    
    # Sum these columns in df and add the result to df_result
    load_reduced[value] = load[cols].sum(axis=1)

In [12]:
sorted_columns = sorted(load_reduced.columns, key=lambda x: int(x))
load_reduced = load_reduced[sorted_columns]


In [13]:
load_reduced

,1,2,3,4,5,6,7
Index,,,,,,,
1,12949.417,14087.040,2334,42510.0,21962.0,13687.066,1851
2,11973.268,13042.870,2176,40912.0,21410.0,13341.066,1775
3,11250.215,12423.089,2004,40153.0,21337.0,13187.066,1735
4,10812.445,11977.669,1864,39976.0,21404.0,13227.066,1699
5,10722.440,11805.469,1818,40403.0,22266.0,13519.066,1689
...,...,...,...,...,...,...,...
8780,18691.857,19822.550,3854,50385.0,28249.0,18125.274,2105
8781,18328.295,19595.190,3789,48999.0,27586.0,18202.274,2084
8782,17809.728,19296.920,3626,47521.0,26256.0,17982.274,2023


In [18]:
load_reduced.to_csv('Processed Data/Load.csv')

# Thermal generator processing

In [3]:
genlist = pd.read_csv('Public Data/GeneratorList.csv',header=2, encoding='ISO-8859-1', usecols=['Name','SubType','Area Name','InitialDispatch(MW)','ServiceStatus','DevStatus'])
thermalgenlist = pd.read_csv('Public Data/Thermal_General_Info.csv',header=1, encoding='ISO-8859-1', usecols=['GeneratorName','MustRun','Fuel Name','MinimumDownTime(hr)','MinimumUpTime(hr)','RampUp Rate(MW/minute)','RampDn Rate(MW/minute)','Startup Cost Fixed($)','Startup Fuel Name','StartFuel(MMBTu)','VOM Cost'])
thermalgencurve = pd.read_csv('Public Data/Thermal_IOCurve_Info.csv',header=1, encoding='ISO-8859-1',usecols=['Generator Name','If Use Generic IO Curve','Generic IO Curve Name','IONumBlock','IOMaxCap(MW)','IOMinCap(MW)','MinInput(MMBTu)','IncCap2(MW)','IncHR2(MMBTu/MWh)','IncCap3(MW)','IncHR3(MMBTu/MWh)','IncCap4(MW)','IncHR4(MMBTu/MWh)','IncCap5(MW)','IncHR5(MMBTu/MWh)','IncCap6(MW)','IncHR6(MMBTu/MWh)'])
fuelcost = pd.read_csv('Public Data/FuelCost_Monthly.csv',header=1, encoding='ISO-8859-1')
thermalgencurve = thermalgencurve[~thermalgencurve['If Use Generic IO Curve']]


In [4]:
# attach zone info to genlist
genlist['Zone'] = genlist['Area Name'].map(region.set_index('Area Name')['Zone'])

In [5]:
# calculate average fuel cost and reduce the dataframe to two columns ['Fuel Name','Fuel Cost($/MMBTu)']
fuelcost['Fuel Cost($/MMBTu)']=fuelcost.iloc[:, 3:15].mean(axis=1)
fuelcost = fuelcost.drop(fuelcost.columns[1:15], axis=1)

In [6]:
thermalgen = pd.merge(thermalgencurve, genlist, left_on='Generator Name', right_on='Name', how='inner')
thermalgen = pd.merge(thermalgen, thermalgenlist, left_on='Generator Name', right_on='GeneratorName', how='inner')

In [7]:
thermalgen = pd.merge(thermalgen, fuelcost, left_on='Fuel Name', right_on='Fuel Name', how='inner')
thermalgen = pd.merge(thermalgen, fuelcost, left_on='Startup Fuel Name', right_on='Fuel Name', how='inner', suffixes=(None, '_Start'))

In [8]:
# cost calculation base on fuel prices
thermalgen['Start Up Cost($)'] = thermalgen['Startup Cost Fixed($)'] + thermalgen['StartFuel(MMBTu)'] * thermalgen['Fuel Cost($/MMBTu)_Start']
thermalgen['NoLoadCost($)'] = thermalgen['MinInput(MMBTu)'] * thermalgen['Fuel Cost($/MMBTu)']
thermalgen['IncCost2($/MW)'] = thermalgen['IncHR2(MMBTu/MWh)'] * thermalgen['Fuel Cost($/MMBTu)']
thermalgen['IncCost3($/MW)'] = thermalgen['IncHR3(MMBTu/MWh)'] * thermalgen['Fuel Cost($/MMBTu)']
thermalgen['IncCost4($/MW)'] = thermalgen['IncHR4(MMBTu/MWh)'] * thermalgen['Fuel Cost($/MMBTu)']
thermalgen['IncCost5($/MW)'] = thermalgen['IncHR5(MMBTu/MWh)'] * thermalgen['Fuel Cost($/MMBTu)']
thermalgen['IncCost6($/MW)'] = thermalgen['IncHR6(MMBTu/MWh)'] * thermalgen['Fuel Cost($/MMBTu)']

In [27]:
thermalgen['IncCost2($/MW)'].max()

978.8601599999998

In [24]:
thermalgen = thermalgen[(thermalgen['DevStatus'] == 'Existing')&(thermalgen['ServiceStatus'] == '#TRUE#')]

In [27]:
thermalgen['MustRun'] = thermalgen['MustRun'].astype(int)

In [51]:
thermalgen.loc[(thermalgen['InitialDispatch(MW)'] == 0) & (thermalgen['MustRun'] == 1), 'InitialDispatch(MW)'] = thermalgen['IOMinCap(MW)']

In [55]:
thermalgen.to_csv('Processed Data/ThermalGen_Full_C.csv')

In [57]:
columns_to_drop = ['If Use Generic IO Curve', 'Generic IO Curve Name','IONumBlock', 'MinInput(MMBTu)',
                   'IncHR2(MMBTu/MWh)', 'IncHR3(MMBTu/MWh)', 'IncHR4(MMBTu/MWh)', 'IncHR5(MMBTu/MWh)','IncHR6(MMBTu/MWh)',
                  'Name', 'GeneratorName', 'Fuel Name', 'Startup Cost Fixed($)', 'Startup Fuel Name', 'StartFuel(MMBTu)',
                  'Fuel Cost($/MMBTu)', 'Fuel Name_Start', 'Fuel Cost($/MMBTu)_Start']
thermalgen = thermalgen.drop(columns=columns_to_drop)

In [59]:
mask = thermalgen['InitialDispatch(MW)'] != 0
thermalgen.loc[mask, 'InitialDispatch(MW)'] = thermalgen.loc[mask, 'InitialDispatch(MW)'].clip(lower=thermalgen.loc[mask, 'IOMinCap(MW)'], upper=thermalgen.loc[mask, 'IOMaxCap(MW)'])

In [60]:
thermalgen['Initial Status'] = (thermalgen['InitialDispatch(MW)'] != 0).astype(int)

In [61]:
thermalgen.to_csv('Processed Data/ThermalGen_C.csv')

# Thermal generator map

In [62]:
thermalgenzone = pd.read_csv('Processed Data/ThermalGen_C.csv', usecols=['Generator Name','Zone'])

In [63]:
thermalgenmap = pd.get_dummies(thermalgenzone["Zone"], dtype=int)
thermalgenmap.to_csv('Processed Data/ThermalGenMap_C.csv')

# Wind  processing

In [14]:
genlist = pd.read_csv('Public Data/GeneratorList.csv',header=2, encoding='ISO-8859-1', usecols=['Name','SubType','Area Name','InitialDispatch(MW)','DevStatus'])
hourlygenlist = pd.read_csv('Public Data/HourlyResource_General.csv',header=2, encoding='ISO-8859-1')

In [15]:
# attach zone info to genlist
genlist['Zone'] = genlist['Area Name'].map(region.set_index('Area Name')['Zone'])
windlist = hourlygenlist[hourlygenlist['Type'] == 'Wind']
windlist = pd.merge(windlist, genlist, left_on='GeneratorName', right_on='Name', how='inner')

In [16]:
columns_to_drop = ['GeneratorKey','Type','CommitmentShapeID','CommitmentMultiplier','DispatchShapeID','DispatchShapeName',
                  'DispatchMultiplier','CommitmentVariation(%)','CommitmentPdf','DispatchVariation(%)','DispatchPdf',
                   'Spillage','IfEnableMultiBlock','ModelType','IfUseInvertLoadingRatio','InvertLoadingRatio','TargetCapacityFactor','Name']
windlist = windlist.drop(columns=columns_to_drop)

In [17]:
# windlist = windlist[windlist['Commission Date'] <= '#2024-01-01#']
windlist = windlist[windlist['DevStatus'] == 'Existing']

In [19]:
sum(windlist[(windlist['Zone']==1) | (windlist['Zone']==2)| (windlist['Zone']==3)]['Capacity(MW)'])

6485.120000000001

In [41]:
windlist.to_csv('Processed Data/Wind_C.csv')

In [20]:
windlist.iloc[0]

GeneratorName           70724_ADS30_Spring Canyon 2-3
CommitmentShapeName    WT_Spring Canyon2WT32 A-1_[18]
Capacity(MW)                                     60.0
SubType                                    WT-Onshore
DevStatus                                    Existing
Area Name                                        PSCO
InitialDispatch(MW)                               0.0
Zone                                                5
Name: 3, dtype: object

# Wind  profile

In [21]:
windshapelist = pd.read_csv('Processed Data/Wind_C.csv', usecols=['GeneratorName','CommitmentShapeName','Capacity(MW)'])
windcruve = pd.read_csv('Public Data/Hourly Profiles in CSV Format/Hourly Profile_Wind.csv',header=0, skiprows=list(range(8786, 8796)))

/var/folders/f2/rxpbf6rd2wjbpqry4k3rs6fr0000gn/T/ipykernel_9216/1033560778.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  windcruve = pd.read_csv('Public Data/Hourly Profiles in CSV Format/Hourly Profile_Wind.csv',header=0, skiprows=list(range(8786, 8796)))


In [22]:
windcruve = windcruve.drop(index=0)

In [23]:
windcruve.columns = [re.sub(r'\.dat:.*', '', col_name) for col_name in windcruve.columns]


In [24]:
windprofile = pd.DataFrame()
for index, row in windshapelist.iterrows():
    generator_name = row['GeneratorName']
    wind_profile_name = row['CommitmentShapeName']
    capacity = row['Capacity(MW)']
    wind_profile = windcruve[wind_profile_name]
    adjusted_profile = wind_profile * capacity
    windprofile[generator_name] = adjusted_profile

/var/folders/f2/rxpbf6rd2wjbpqry4k3rs6fr0000gn/T/ipykernel_9216/1997807084.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  windprofile[generator_name] = adjusted_profile
/var/folders/f2/rxpbf6rd2wjbpqry4k3rs6fr0000gn/T/ipykernel_9216/1997807084.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  windprofile[generator_name] = adjusted_profile
/var/folders/f2/rxpbf6rd2wjbpqry4k3rs6fr0000gn/T/ipykernel_9216/1997807084.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert`

In [25]:
windprofile.to_csv('Processed Data/WindProfile_C.csv')

# Wind  map

In [29]:
windzone = pd.read_csv('Processed Data/Wind_C.csv', usecols=['GeneratorName','Zone'])

In [30]:
windmap = pd.get_dummies(windzone["Zone"], dtype=int)
windmap.to_csv('Processed Data/WindMap_C.csv')

In [40]:
windprofile = pd.read_csv('Processed Data/WindProfile_C.csv').iloc[:,1:].values
windmap = pd.read_csv('Processed Data/WindMap_C.csv').iloc[:,1:].values

windzone = pd.DataFrame(np.matmul(windprofile,windmap))
windzone.to_csv('Processed Data/WindZone_C.csv')

# Solar processing

In [50]:
genlist = pd.read_csv('Public Data/GeneratorList.csv',header=2, encoding='ISO-8859-1', usecols=['Name','SubType','Area Name','InitialDispatch(MW)','Commission Date','DevStatus'])
hourlygenlist = pd.read_csv('Public Data/HourlyResource_General.csv',header=2, encoding='ISO-8859-1')

In [51]:
genlist['Zone'] = genlist['Area Name'].map(region.set_index('Area Name')['Zone'])
solarlist = hourlygenlist[hourlygenlist['Type'] == 'Solar']
solarlist = pd.merge(solarlist, genlist, left_on='GeneratorName', right_on='Name', how='inner')

In [52]:
columns_to_drop = ['GeneratorKey','Type','CommitmentShapeID','CommitmentMultiplier','DispatchShapeID','DispatchShapeName',
                  'DispatchMultiplier','CommitmentVariation(%)','CommitmentPdf','DispatchVariation(%)','DispatchPdf',
                   'Spillage','IfEnableMultiBlock','ModelType','IfUseInvertLoadingRatio','InvertLoadingRatio','TargetCapacityFactor','Name']
solarlist = solarlist.drop(columns=columns_to_drop)

In [53]:
# solarlist = solarlist[solarlist['Commission Date'] <= '#2023-01-01#']
solarlist = solarlist[solarlist['DevStatus'] == 'Existing']

In [54]:
sum(solarlist[(solarlist['Zone']==1) | (solarlist['Zone']==2)]['Capacity(MW)'])

16006.899999999996

In [55]:
solarlist.to_csv('Processed Data/Solar_C.csv')

# Solar profile


In [56]:
solarshapelist = pd.read_csv('Processed Data/Solar_C.csv', usecols=['GeneratorName','CommitmentShapeName','Capacity(MW)'])
solarcruve1 = pd.read_csv('Public Data/Hourly Profiles in CSV Format/Hourly Profile_Solar1.csv',header=0, skiprows=list(range(8786, 8796)))
solarcruve2 = pd.read_csv('Public Data/Hourly Profiles in CSV Format/Hourly Profile_Solar2.csv',header=0, skiprows=list(range(8786, 8796)))
solarcruve3 = pd.read_csv('Public Data/Hourly Profiles in CSV Format/Hourly Profile_BTM Solar_DPV.csv',header=0, skiprows=list(range(8786, 8796)))

C:\Users\wenmi\AppData\Local\Temp\ipykernel_13196\3958611362.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  solarcruve1 = pd.read_csv('Public Data/Hourly Profiles in CSV Format/Hourly Profile_Solar1.csv',header=0, skiprows=list(range(8786, 8796)))
C:\Users\wenmi\AppData\Local\Temp\ipykernel_13196\3958611362.py:3: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  solarcruve2 = pd.read_csv('Public Data/Hourly Profiles in CSV Format/Hourly Profile_Solar2.csv',header=0, skiprows=list(range(8786, 8796)))
C:\Users\wenmi\AppData\Local\Temp\ipykernel_13196\3958611362.py:4: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  solarcruve3 = pd.read_csv('Public Data/Hourly Profiles in CSV Format/Hourly Profile_BTM Solar_DPV.csv',header=0, skiprows=list(range(8786, 8796)))


In [57]:
solarcruve1 = solarcruve1.drop(index=0)
solarcruve2 = solarcruve2.drop(index=0)
solarcruve3 = solarcruve3.drop(index=0)

In [58]:
solarcruve1.columns = [re.sub(r'\.DAT: 2032$|\.dat: 2032$', '', col_name) for col_name in solarcruve1.columns]
solarcruve2.columns = [re.sub(r'\.DAT: 2032$|\.dat: 2032$', '', col_name) for col_name in solarcruve2.columns]
solarcruve3.columns = [re.sub(r'\.DAT: 2032$|\.dat: 2032$', '', col_name) for col_name in solarcruve3.columns]


solarcruve = pd.merge(solarcruve1, solarcruve2, on='Index')
solarcruve = pd.merge(solarcruve, solarcruve3, on='Index')



In [59]:
solarprofile = pd.DataFrame()
for index, row in solarshapelist.iterrows():
    generator_name = row['GeneratorName']
    solar_profile_name = row['CommitmentShapeName']
    capacity = row['Capacity(MW)']
    solar_profile = solarcruve[solar_profile_name]
    adjusted_profile = solar_profile * capacity
    solarprofile[generator_name] = adjusted_profile

C:\Users\wenmi\AppData\Local\Temp\ipykernel_13196\1595787671.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  solarprofile[generator_name] = adjusted_profile
C:\Users\wenmi\AppData\Local\Temp\ipykernel_13196\1595787671.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  solarprofile[generator_name] = adjusted_profile
C:\Users\wenmi\AppData\Local\Temp\ipykernel_13196\1595787671.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.

C:\Users\wenmi\AppData\Local\Temp\ipykernel_13196\1595787671.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  solarprofile[generator_name] = adjusted_profile
C:\Users\wenmi\AppData\Local\Temp\ipykernel_13196\1595787671.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  solarprofile[generator_name] = adjusted_profile
C:\Users\wenmi\AppData\Local\Temp\ipykernel_13196\1595787671.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.

In [60]:
solarprofile.to_csv('Processed Data/SolarProfile_C.csv')

# Solar  map

In [42]:
solarzone = pd.read_csv('Processed Data/Solar_C.csv', usecols=['GeneratorName','Zone'])
solarmap = pd.get_dummies(solarzone["Zone"], dtype=int)
solarmap.to_csv('Processed Data/SolarMap_C.csv')

In [43]:
solarprofile = pd.read_csv('Processed Data/SolarProfile_C.csv').iloc[:,1:].values
solarmap = pd.read_csv('Processed Data/SolarMap_C.csv').iloc[:,1:].values

solarzone = pd.DataFrame(np.matmul(solarprofile,solarmap))
solarzone.to_csv('Processed Data/SolarZone_C.csv')

# Hydro  processing

In [62]:
genlist = pd.read_csv('Public Data/GeneratorList.csv',header=2, encoding='ISO-8859-1', usecols=['Name','SubType','Area Name','MinCap(MW)','MaxCap(MW)','InitialDispatch(MW)','DevStatus'])
genlist['Zone'] = genlist['Area Name'].map(region.set_index('Area Name')['Zone'])
hydrolist = genlist.loc[genlist['SubType'].isin(['Hydro', 'HydroRPS'])]

In [63]:
hydrolist = hydrolist[hydrolist['DevStatus'] == 'Existing']

In [64]:
hydrolist.to_csv('Processed Data/Hydro_C.csv')

In [65]:
hydrocruve = pd.read_csv('Public Data/Hourly Profiles in CSV Format/Hourly Profile_Hydro by Area.csv',header=0, skiprows=list(range(8786, 8796)))
hydrocruve = hydrocruve.drop(index=0)
hydrocruve.columns = [re.sub('HY_(.*?)_2018.dat: 2032', r'\1', col_name) for col_name in hydrocruve.columns]


In [66]:
hydroprofile = pd.DataFrame()
for index, row in hydrolist.iterrows():
    generator_name = row['Name']
    hydro_profile_name = row['Area Name']
    capacity = row['MaxCap(MW)']
    if hydro_profile_name == 'SRP':
        hydro_profile = hydrocruve['PNM']
    elif hydro_profile_name in {'CISC','CISD'}:
        hydro_profile = hydrocruve['CISC_CISD']
    elif hydro_profile_name in {'CIPV','CIPB'}:
        hydro_profile = hydrocruve['CIPV_CIPB']
    elif hydro_profile_name in {'CHPD','DOPD'}:
        hydro_profile = hydrocruve['GCPD']
    elif hydro_profile_name in {'PSCO','IPMV','IPTV','IPFE'}:
        hydro_profile = hydrocruve['PSCO_IPMV_IPTV_IPFE']
    elif hydro_profile_name == 'WAUW':
        hydro_profile = hydrocruve['WACM']
    elif hydro_profile_name in {'WALC','SPPC'}:
        hydro_profile = hydrocruve['PAUT']
    elif hydro_profile_name == 'TH_Mead':
        hydro_profile = hydrocruve['NEVP']
    else: 
        hydro_profile = hydrocruve[hydro_profile_name]
    adjusted_profile = hydro_profile * capacity
    hydroprofile[generator_name] = adjusted_profile

C:\Users\wenmi\AppData\Local\Temp\ipykernel_13196\1577005624.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  hydroprofile[generator_name] = adjusted_profile
C:\Users\wenmi\AppData\Local\Temp\ipykernel_13196\1577005624.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  hydroprofile[generator_name] = adjusted_profile
C:\Users\wenmi\AppData\Local\Temp\ipykernel_13196\1577005624.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performan

C:\Users\wenmi\AppData\Local\Temp\ipykernel_13196\1577005624.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  hydroprofile[generator_name] = adjusted_profile
C:\Users\wenmi\AppData\Local\Temp\ipykernel_13196\1577005624.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  hydroprofile[generator_name] = adjusted_profile
C:\Users\wenmi\AppData\Local\Temp\ipykernel_13196\1577005624.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performan

C:\Users\wenmi\AppData\Local\Temp\ipykernel_13196\1577005624.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  hydroprofile[generator_name] = adjusted_profile
C:\Users\wenmi\AppData\Local\Temp\ipykernel_13196\1577005624.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  hydroprofile[generator_name] = adjusted_profile
C:\Users\wenmi\AppData\Local\Temp\ipykernel_13196\1577005624.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performan

In [67]:
hydroprofile.to_csv('Processed Data/HydroProfile_C.csv')

# Hydro  map


In [68]:
hydrozone = pd.read_csv('Processed Data 2 Zone/Hydro_C.csv', usecols=['Name','Zone'])
hydromap = pd.get_dummies(hydrozone["Zone"], dtype=int)
hydromap.to_csv('Processed Data 2 Zone/HydroMap_C.csv')

# Storage  processing

In [3]:
genlist = pd.read_csv('Public Data/GeneratorList.csv',header=2, encoding='ISO-8859-1', usecols=['Name','SubType','Area Name','MinCap(MW)','MaxCap(MW)','InitialDispatch(MW)','DevStatus'])
genlist['Zone'] = genlist['Area Name'].map(region.set_index('Area Name')['Zone'])

In [4]:
storagelist = genlist.loc[genlist['SubType'].isin(['PS-Hydro', 'PS-HydroRPS','Battery Storage'])]
storagelist['Efficiency'] = np.where(storagelist['SubType'].isin(['PS-Hydro', 'PS-HydroRPS']), 0.80, 
                            np.where(storagelist['SubType'] == 'Battery Storage', 0.90, np.nan))
storagelist['Duration(h)'] = np.where(storagelist['SubType'].isin(['PS-Hydro', 'PS-HydroRPS']), 12, 
                            np.where(storagelist['SubType'] == 'Battery Storage', 2, np.nan))
storagelist['MaxCap(MWh)'] = storagelist['MaxCap(MW)'] * storagelist['Duration(h)']


C:\Users\wenmi\AppData\Local\Temp\ipykernel_12372\549637819.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  storagelist['Efficiency'] = np.where(storagelist['SubType'].isin(['PS-Hydro', 'PS-HydroRPS']), 0.80,
C:\Users\wenmi\AppData\Local\Temp\ipykernel_12372\549637819.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  storagelist['Duration(h)'] = np.where(storagelist['SubType'].isin(['PS-Hydro', 'PS-HydroRPS']), 12,
C:\Users\wenmi\AppData\Local\Temp\ipykernel_12372\549637819.py:6: SettingWithCopyWarnin

In [5]:
storagelist = storagelist[storagelist['DevStatus'] == 'Existing']

In [6]:
storagelist.to_csv('Processed Data/Storage_C.csv')

# Storage  map


In [8]:
storagezone = pd.read_csv('Processed Data/Storage_C.csv', usecols=['Name','Zone'])
storagemap = pd.get_dummies(storagezone["Zone"], dtype=int)
storagemap.to_csv('Processed Data/StorageMap_C.csv')